In [1]:
import chipwhisperer as cw
scope = cw.scope()

/home/nico/miniconda3/envs/simon/lib/python3.13/site-packages/chipwhisperer/capture/trace/TraceWhisperer.py:31: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources # type: ignore


In [63]:
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()

scope.clock.adc_freq                     changed from 29538471                  to 29538459                 
scope.clock.adc_rate                     changed from 29538471.0                to 29538459.0               


In [94]:
%%bash
cd ../chipwhisperer/firmware/mcu/simpleserial-simon/
make PLATFORM=CWLITEARM CRYPTO_TARGET=SIMON CRYPTO_OPTIONS=NONE

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:13.2.rel1-2) 13.2.1 20231009
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CWLITEARM 
.
Compiling:
-en     simpleserial-simon.c ...


simpleserial-simon.c: In function 'get_key':
simpleserial-simon.c:58:1: warning: control reaches end of non-void function [-Wreturn-type]
   58 | }
      | ^


-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/hal.c ...
-e Done!
.
Compiling:
-en     .././hal//stm32f3/stm32f3_hal.c ...
-e Done!
.
Compiling:
-en     .././hal//stm32f3/stm32f3_hal_lowlevel.c ...
-e Done!
.
Compiling:
-en     .././hal//stm32f3/stm32f3_sysmem.c ...
-e Done!
.
Compiling:
-en     .././crypto/simon64_128.c ...
-e Done!
.
Assembling: .././hal//stm32f3/stm32f3_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWLITEARM/stm32f3_startup.lst -I.././simpleserial/ -I.././hal/ -I.././hal/ -I.././hal//stm32f3 -I.././hal//stm32f3/CMSIS -I.././hal//stm32f3/CMSIS/core -I.././hal//stm32f3/CMSIS/device -I.././hal//stm32f4/Legacy -I.././simpleserial/ -I.././crypto/ .././hal//stm32f3/stm32f3_startup.S -o objdir-CWLITEARM/stm32f3_startup.o
.
LINKING:
-en     simpleserial-simon-CWLITEARM.elf ...
Memory 

In [67]:
scope.dis()
target.dis()
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 318163443                 to 358767902                
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.clkgen_div                   changed from 1                         to 26                       
scope.clock.clkgen_freq                  changed from 192000000.0               to 7384615.384615385        
scope.io.tio1      

In [95]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "../chipwhisperer/firmware/mcu/simpleserial-simon/simpleserial-simon-CWLITEARM.hex")

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4987 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4987 bytes


In [ ]:
key = bytearray([0x1B, 0x1A, 0x19, 0x18, 0x13, 0x12, 0x11, 0x10, 0x0B, 0x0A, 0x09, 0x08, 0x03, 0x02, 0x01, 0x00])
pt = bytearray([0x65, 0x6B, 0x69, 0x6C, 0x20, 0x64, 0x6E, 0x75])

target.simpleserial_write('k', key)
res: bytearray = target.simpleserial_read('r', 1)
# print("len: {}".format(res.hex()))

target.simpleserial_write('p', pt)
ct: bytearray = target.simpleserial_read('r', 8)

print("Ciphertext: {}".format(ct.hex()))

(ChipWhisperer Target WARNING|File SimpleSerial.py:447) Unexpected start to command: z


AttributeError: 'NoneType' object has no attribute 'hex'